# Belgian IBAN Numbers

## Introduction

The function `clean_be_iban()` cleans a column containing Belgian IBAN (International Bank Account Number) strings, and standardizes them in a given format. The function `validate_be_iban()` validates either a single Belgian IBAN strings, a column of Belgian IBAN strings or a DataFrame of Belgian IBAN strings, returning `True` if the value is valid, and `False` otherwise.

Belgian IBAN strings can be converted to the following formats via the `output_format` parameter:

* `compact`: only number strings without any seperators or whitespace, like "BE32123456789002"
* `standard`: Belgian IBAN strings with proper whitespace in the proper places, like "BE32 1234 5678 9002"
* `bic`: return the BIC for the bank that this number refers to, like "CTBKBEBX".

Invalid parsing is handled with the `errors` parameter:

* `coerce` (default): invalid parsing will be set to NaN
* `ignore`: invalid parsing will return the input
* `raise`: invalid parsing will raise an exception

The following sections demonstrate the functionality of `clean_be_iban()` and `validate_be_iban()`. 

### An example dataset containing Belgian IBAN strings

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame(
    {
        "be_iban": [
            "BE32 123-4567890-02",
            "BE41091811735141",             # incorrect national check digits
            "BE83138811735115",             # unknown bank code
            "GR1601101050000010547023795",  # not a Belgian IBAN
            "BE 48 3200 7018 4927",
            "hello",
            np.nan,
            "NULL",
        ], 
        "address": [
            "123 Pine Ave.",
            "main st",
            "1234 west main heights 57033",
            "apt 1 789 s maple rd manhattan",
            "robie house, 789 north main street",
            "1111 S Figueroa St, Los Angeles, CA 90015",
            "(staples center) 1111 S Figueroa St, Los Angeles",
            "hello",
        ]
    }
)
df

,be_iban,address
0,BE32 123-4567890-02,123 Pine Ave.
1,BE41091811735141,main st
2,BE83138811735115,1234 west main heights 57033
3,GR1601101050000010547023795,apt 1 789 s maple rd manhattan
4,BE 48 3200 7018 4927,"robie house, 789 north main street"
5,hello,"1111 S Figueroa St, Los Angeles, CA 90015"
6,NaN,"(staples center) 1111 S Figueroa St, Los Angeles"
7,NULL,hello


## 1. Default `clean_be_iban`

By default, `clean_be_iban` will clean be_iban strings and output them in the standard format with proper separators.

In [2]:
from dataprep.clean import clean_be_iban
clean_be_iban(df, column = "be_iban")

,be_iban,address,be_iban_clean
0,BE32 123-4567890-02,123 Pine Ave.,BE32 1234 5678 9002
1,BE41091811735141,main st,NaN
2,BE83138811735115,1234 west main heights 57033,NaN
3,GR1601101050000010547023795,apt 1 789 s maple rd manhattan,NaN
4,BE 48 3200 7018 4927,"robie house, 789 north main street",BE48 3200 7018 4927
5,hello,"1111 S Figueroa St, Los Angeles, CA 90015",NaN
6,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
7,NULL,hello,NaN


## 2. Output formats

This section demonstrates the output parameter.

### `standard` (default)

In [3]:
clean_be_iban(df, column = "be_iban", output_format="standard")

,be_iban,address,be_iban_clean
0,BE32 123-4567890-02,123 Pine Ave.,BE32 1234 5678 9002
1,BE41091811735141,main st,NaN
2,BE83138811735115,1234 west main heights 57033,NaN
3,GR1601101050000010547023795,apt 1 789 s maple rd manhattan,NaN
4,BE 48 3200 7018 4927,"robie house, 789 north main street",BE48 3200 7018 4927
5,hello,"1111 S Figueroa St, Los Angeles, CA 90015",NaN
6,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
7,NULL,hello,NaN


### `compact`

In [4]:
clean_be_iban(df, column = "be_iban", output_format="compact")

,be_iban,address,be_iban_clean
0,BE32 123-4567890-02,123 Pine Ave.,BE32123456789002
1,BE41091811735141,main st,NaN
2,BE83138811735115,1234 west main heights 57033,NaN
3,GR1601101050000010547023795,apt 1 789 s maple rd manhattan,NaN
4,BE 48 3200 7018 4927,"robie house, 789 north main street",BE48320070184927
5,hello,"1111 S Figueroa St, Los Angeles, CA 90015",NaN
6,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
7,NULL,hello,NaN


### `bic`

In [5]:
clean_be_iban(df, column = "be_iban", output_format="bic")

,be_iban,address,be_iban_clean
0,BE32 123-4567890-02,123 Pine Ave.,CTBKBEBX
1,BE41091811735141,main st,NaN
2,BE83138811735115,1234 west main heights 57033,NaN
3,GR1601101050000010547023795,apt 1 789 s maple rd manhattan,NaN
4,BE 48 3200 7018 4927,"robie house, 789 north main street",BBRUBEBB
5,hello,"1111 S Figueroa St, Los Angeles, CA 90015",NaN
6,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
7,NULL,hello,NaN


## 3. `inplace` parameter

This deletes the given column from the returned DataFrame. 
A new column containing cleaned Belgian IBAN strings is added with a title in the format `"{original title}_clean"`.

In [6]:
clean_be_iban(df, column="be_iban", inplace=True)

,be_iban_clean,address
0,BE32 1234 5678 9002,123 Pine Ave.
1,NaN,main st
2,NaN,1234 west main heights 57033
3,NaN,apt 1 789 s maple rd manhattan
4,BE48 3200 7018 4927,"robie house, 789 north main street"
5,NaN,"1111 S Figueroa St, Los Angeles, CA 90015"
6,NaN,"(staples center) 1111 S Figueroa St, Los Angeles"
7,NaN,hello


## 4. `errors` parameter

### `coerce` (default)

In [7]:
clean_be_iban(df, "be_iban", errors="coerce")

,be_iban,address,be_iban_clean
0,BE32 123-4567890-02,123 Pine Ave.,BE32 1234 5678 9002
1,BE41091811735141,main st,NaN
2,BE83138811735115,1234 west main heights 57033,NaN
3,GR1601101050000010547023795,apt 1 789 s maple rd manhattan,NaN
4,BE 48 3200 7018 4927,"robie house, 789 north main street",BE48 3200 7018 4927
5,hello,"1111 S Figueroa St, Los Angeles, CA 90015",NaN
6,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
7,NULL,hello,NaN


### `ignore`

In [8]:
clean_be_iban(df, "be_iban", errors="ignore")

,be_iban,address,be_iban_clean
0,BE32 123-4567890-02,123 Pine Ave.,BE32 1234 5678 9002
1,BE41091811735141,main st,BE41091811735141
2,BE83138811735115,1234 west main heights 57033,BE83138811735115
3,GR1601101050000010547023795,apt 1 789 s maple rd manhattan,GR1601101050000010547023795
4,BE 48 3200 7018 4927,"robie house, 789 north main street",BE48 3200 7018 4927
5,hello,"1111 S Figueroa St, Los Angeles, CA 90015",hello
6,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
7,NULL,hello,NaN


## 4. `validate_be_iban()`

`validate_be_iban()` returns `True` when the input is a valid Belgian IBAN. Otherwise it returns `False`.

The input of `validate_be_iban()` can be a string, a Pandas DataSeries, a Dask DataSeries, a Pandas DataFrame and a dask DataFrame.

When the input is a string, a Pandas DataSeries or a Dask DataSeries, user doesn't need to specify a column name to be validated. 

When the input is a Pandas DataFrame or a dask DataFrame, user can both specify or not specify a column name to be validated. If user specify the column name, `validate_be_iban()` only returns the validation result for the specified column. If user doesn't specify the column name, `validate_be_iban()` returns the validation result for the whole DataFrame.

In [9]:
from dataprep.clean import validate_be_iban
print(validate_be_iban("BE32 123-4567890-02"))
print(validate_be_iban("BE41091811735141"))
print(validate_be_iban("BE83138811735115"))
print(validate_be_iban("GR1601101050000010547023795"))
print(validate_be_iban("004085616"))
print(validate_be_iban("hello"))
print(validate_be_iban(np.nan))
print(validate_be_iban("NULL"))

True
False
False
False
False
False
False
False


### Series

In [10]:
validate_be_iban(df["be_iban"])

0     True
1    False
2    False
3    False
4     True
5    False
6    False
7    False
Name: be_iban, dtype: bool

### DataFrame + Specify Column

In [11]:
validate_be_iban(df, column="be_iban")

0     True
1    False
2    False
3    False
4     True
5    False
6    False
7    False
Name: be_iban, dtype: bool

### Only DataFrame

In [12]:
validate_be_iban(df)

,be_iban,address
0,True,False
1,False,False
2,False,False
3,False,False
4,True,False
5,False,False
6,False,False
7,False,False
